<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Template" data-toc-modified-id="Template-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Template</a></span><ul class="toc-item"><li><span><a href="#Functions-useful" data-toc-modified-id="Functions-useful-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Functions useful</a></span></li><li><span><a href="#Run-all-files-from-directory" data-toc-modified-id="Run-all-files-from-directory-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Run all files from directory</a></span><ul class="toc-item"><li><span><a href="#Values-I-want" data-toc-modified-id="Values-I-want-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Values I want</a></span></li></ul></li></ul></li></ul></div>

# Template

## Functions useful

In [1]:
import pandas as pd
import numpy as np
import xlrd
import string
import os

In [2]:
def read_file(IO):
    return pd.read_excel(io=IO)

In [3]:
#稳态工况修正
def clean_data(sheet,ty):
    spec_name = get_spec_name(ty)
    data = sheet.loc[:,['供水温度/℃','室内平均温度',spec_name,'室外温度/℃','供热量']]
    # print("the data format is:\n{0}".format(data))
    return data

# #稳态工况平均值
def ave(data,start,end):
    return sum(data[start:end])/(end-start)

In [4]:
#热量修正1：总供热量(用于末端自身特征)
def repair_heat1(temp_suwat,temp_rewat,volume_suwat,error_wat):
    c_w = 4200  #J/kgK
    heat_suwat1 = [c_w*volume_suwat[i]*(temp_suwat[i]-temp_rewat[i]+error_wat)/3600 for i in range(len(volume_suwat))]
    return heat_suwat1

#热量修正2：总供热量（用于末端对比）
def repair_heat2(heat_suwat,ita_env):
    return [x*ita_env for x in heat_suwat]

In [5]:
#计算操作温度
def operative(temp_inair,temp_envelope):
    x = np.minimum(len(temp_inair),len(temp_envelope))
    temp_ope = [(temp_inair[i]+temp_envelope[i])/2 for i in range(x)]
    return temp_ope

In [6]:
#计算末端表面和室内环境稳定时间

# 计算逐时温度与初始温度之差
def sub(data):
    sub_results = []
    for i in range(len(data)):
        if i ==(len(data)-1):
            break
        else:
            sub_results.append(data[i+1]-data[0])
    return sub_results


#每时刻温升/最大温升，从这里面找到>95%的那个点，认为是达到了稳态时间。
def find_stable_time(data,max_tempdif):
    for i in range(len(data)):
        if data[i] > 0.95 * max_tempdif:
            return i+1
    return len(data)

In [7]:
#计算末端的R1,R2,C
#稳态工况（稳态时间，10）：R1 =ave【（供水温度-末端温度）/水侧供热量】【形成最后数组】
#稳态工况（稳态时间，10）：R2 = ave【（末端温度-（室内空气温度+非供暖围护结构表面温度）/2）/水侧供热量】【形成最后数组】
#启动工况（0，稳态时间）：C = sum（水侧供热量）* 60 /（稳态时间下末端温度-起始工况下末端温度）【形成最后数组】

#计算等效R1
def get_R1(temp_suwat,temp_tersur,heat_suwat2,start_time,end_time):
    sub = [temp_suwat[i] - temp_tersur[i] for i in range(end_time)]
    return np.mean([sub[i] / heat_suwat2[i] for i in range(start_time,end_time)])

#计算等效R2
def get_R2(temp_tersur,temp_ope,heat_suwat2,start_time,end_time):
    sub = [temp_tersur[i] - temp_ope[i] for i in range(end_time)]
    return np.mean([sub[i] / heat_suwat2[i] for i in range(start_time,end_time)])

#计算末端向室内逐时供热量
def get_Q(temp_tersur,temp_ope,R2,end_time):
    sub = [temp_tersur[i] - temp_ope[i] for i in range(end_time)]
    return [x/R2 for x in sub]

#计算等效C
def get_C(heat_store,tempdif_initial_stable_tersur,stable_time_tersur,sta_duration):
    sum_heat = sum(heat_store[i] for i in range(0,stable_time_tersur))*sta_duration 
    return sum_heat/tempdif_initial_stable_tersur


In [8]:
#室内空气温度，1.1m以下空气，1.1m以上空气 到达18℃所用时间【形成最后数组】
def time_to_arrive(data,target_temp):
    for i in range(len(data)):
        if data[i] > target_temp:
            return i
    return len(data)


In [9]:
#用温升算局部空间热量

def temp_up(data):
    len_data = len(data)
    sub_results = []
    for i in range(len_data):
        if i == (len_data-1):
            break
        else:
            sub_results.append(data[i+1] - data[i])
    return sub_results


In [10]:
#用累积温升算局部空间热量

def temp_up_total(data):
    len_data = len(data)
    sub_results = []
    for i in range(len_data):
        if i == (len_data-1):
            break
        else:
            sub_results.append(data[i+1] - data[0])
    return sub_results

In [11]:
#用室内外温差算热量匹配系数
def dynamic_tempdif(data,temp_outair):
    x = np.minimum(len(data),len(temp_outair))
    return [data[i] - temp_outair[i] for i in range(x)]


In [12]:
#计算温度匹配系数
# 问题： 稍微修改了一下代码，应该没问题
def temp_match_coefficient(temp_operative,temp_operative_target,operating_duration):
    temp_initial = temp_operative[0]
    sub = temp_operative_target-temp_initial
    TMC = (sum(temp_operative[i] for i in range(0,operating_duration))-temp_initial*operating_duration)/(sub*operating_duration)
    return TMC

## Run all files from directory



### Values I want

#### 1.整个房间：平均参数

稳态时刻：供水温度、回水温度、末端温度、室内平均空气温度、非供暖表面平均温度、室外平均温度、室内外温差、修正前供热量、修正后供热量、末端稳定时间、室内空气稳定时间、最后1h室内操作温度温升

  1.1 稳态时刻：外窗温度、东墙温度、西墙温度、南墙温度、北墙温度、屋顶温度、地面温度

  1.2 稳态时刻：散热器表面1温度、散热器表面2温度、散热器表面3温度、散热器表面4温度

#### 2.末端时间（传热）特征

R1、R2、C（等效传热热阻、等效散热热阻、等效热容）

#### 3.末端空间特征

启动时刻：zoneA平均温度、zoneB平均温度（1.2m以下为zoneA区域）

稳态时刻：zoneA稳态温度、zoneB稳态温度、1.7-0.1垂直温差、1.1-0.1垂直温差

热量匹配系数：

（1）用温升算： 
 
 A区0.5h热量匹配系数、B区0.5h热量匹配系数
  
  A区启动阶段热量匹配系数、B区启动阶段热量匹配系数

（2）用室内外温差算：

A区稳态热量分配系数（用室内外温差算）、B区稳态热量分配系数（用室内外温差算）

#### 4.节能性

（1）热量：

达到稳定时刻热源累积供热量、末端向室内累积供热量、末端累积蓄热量

稳态时刻热源供热量

（2）火积耗散：

达到稳态时刻总累积火积耗散、热源-末端累积火积耗散、末端-室内累积火积耗散、室内-室外累积火积耗散

稳态时刻总火积耗散、热源-末端火积耗散、末端-室内火积耗散、室内-室外火积耗散

#### 5.舒适性

zoneA上升10℃所用时间，zoneB上升10℃所用时间，室内空气温度上升10℃所用时间

1h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体

2h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体

3h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体

4h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体

5h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体

6h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体

In [13]:
import os 
my_list = os.listdir('Radiator_left_15℃')

In [14]:
list_data1 = []   #存储sheet1
list_data2 = []   #存储sheet2
for f in my_list:
    path = "Radiator_left_15℃/"+f
    print(f)
    list_data1.append((pd.read_excel(path,'末端侧与空气侧',index_col="时间/s"),'末端侧与空气侧_'+f))
    list_data2.append((pd.read_excel(path,'空气分布',index_col="时间/s"),'空气分布_'+f))

211220_R1_60_0.6m.xlsx
211221_R1_60_1.2m.xlsx
211218_R1_60_0.6m.xlsx
220119_R1_50_0m.xlsx


In [15]:
def read1(sheet):
    #读取每一列，动态数据
    temp_suwat = list(sheet['供水温度/℃'])
    temp_rewat = list(sheet['回水温度/℃'])
    temp_inair = list(sheet['室内平均温度'])
    temp_tersur1 = list(sheet['暖气表面1/℃'])
    temp_tersur2 = list(sheet['暖气表面2/℃'])
    temp_tersur3 = list(sheet['暖气表面3/℃'])
    temp_tersur4 = list(sheet['暖气表面4/℃'])
    temp_outair = list(sheet['室外温度/℃'])
    volume_suwat = list(sheet['供水流量kg/h'])
    temp_eastwall1 = list(sheet['西墙低/℃'])
    temp_eastwall2 = list(sheet['西墙高/℃'])
    temp_westwall1 = list(sheet['东墙低/℃'])
    temp_westwall2 = list(sheet['东墙高/℃'])
    temp_northwall = list(sheet['北墙/℃'])
    temp_southwall = list(sheet['南墙/℃'])
    temp_roof = list(sheet['屋顶/℃'])
    temp_window = list(sheet['窗户/℃'])
    temp_floor = list(sheet['地面/℃'])
    return temp_suwat,temp_rewat,temp_inair,temp_tersur1,temp_tersur2,temp_tersur3,temp_tersur4,temp_outair,volume_suwat,temp_eastwall1,temp_eastwall2,temp_westwall1,temp_westwall2,temp_northwall,temp_southwall,temp_roof,temp_window,temp_floor


In [16]:
def read2(sheet):
    time = list(sheet.index)
    temp_door_d01 = list(sheet['门d01/℃'])
    temp_door_d06 = list(sheet['门d06/℃'])
    temp_door_d11 = list(sheet['门d11/℃'])
    temp_door_d17 = list(sheet['门d17/℃'])
    temp_door_d23 = list(sheet['门d23/℃'])
    temp_door_c01 = list(sheet['门c01/℃'])
    temp_door_c06 = list(sheet['门c06/℃'])
    temp_door_c11 = list(sheet['门c11/℃'])
    temp_door_c17 = list(sheet['门c17/℃'])
    temp_door_c23 = list(sheet['门c23/℃'])
    temp_door_b01 = list(sheet['门b01/℃'])
    temp_door_b06 = list(sheet['门b06/℃'])
    temp_door_b11 = list(sheet['门b11/℃'])
    temp_door_b17 = list(sheet['门b17/℃'])
    temp_door_b23 = list(sheet['门b23/℃'])
    temp_door_a01 = list(sheet['门a01/℃'])
    temp_door_a06 = list(sheet['门a06/℃'])
    temp_door_a11 = list(sheet['门a11/℃'])
    temp_door_a17 = list(sheet['门a17/℃'])
    temp_door_a23 = list(sheet['门a23/℃'])
    temp_mid_d01 = list(sheet['中d01/℃'])
    temp_mid_d06 = list(sheet['中d06/℃'])
    temp_mid_d11 = list(sheet['中d11/℃'])
    temp_mid_d17 = list(sheet['中d17/℃'])
    temp_mid_d23 = list(sheet['中d23/℃'])
    temp_mid_c01 = list(sheet['中c01/℃'])
    temp_mid_c06 = list(sheet['中c06/℃'])
    temp_mid_c11 = list(sheet['中c11/℃'])
    temp_mid_c17 = list(sheet['中c17/℃'])
    temp_mid_c23 = list(sheet['中c23/℃'])
    temp_mid_b01 = list(sheet['中b01/℃'])
    temp_mid_b06 = list(sheet['中b06/℃'])
    temp_mid_b11 = list(sheet['中b11/℃'])
    temp_mid_b17 = list(sheet['中b17/℃'])
    temp_mid_b23 = list(sheet['中b23/℃'])
    temp_mid_a01 = list(sheet['中a01/℃'])
    temp_mid_a06 = list(sheet['中a06/℃'])
    temp_mid_a11 = list(sheet['中a11/℃'])
    temp_mid_a17 = list(sheet['中a17/℃'])
    temp_mid_a23 = list(sheet['中a23/℃'])
    temp_window_d01 = list(sheet['窗d01/℃'])
    temp_window_d06 = list(sheet['窗d06/℃'])
    temp_window_d11 = list(sheet['窗d11/℃'])
    temp_window_d17 = list(sheet['窗d17/℃'])
    temp_window_d23 = list(sheet['窗d23/℃'])
    temp_window_c01 = list(sheet['窗c01/℃'])
    temp_window_c06 = list(sheet['窗c06/℃'])
    temp_window_c11 = list(sheet['窗c11/℃'])
    temp_window_c17 = list(sheet['窗c17/℃'])
    temp_window_c23 = list(sheet['窗c23/℃'])
    temp_window_b01 = list(sheet['窗b01/℃'])
    temp_window_b06 = list(sheet['窗b06/℃'])
    temp_window_b11 = list(sheet['窗b11/℃'])
    temp_window_b17 = list(sheet['窗b17/℃'])
    temp_window_b23 = list(sheet['窗b23/℃'])
    temp_window_a01 = list(sheet['窗a01/℃'])
    temp_window_a06 = list(sheet['窗a06/℃'])
    temp_window_a11 = list(sheet['窗a11/℃'])
    temp_window_a17 = list(sheet['窗a17/℃'])
    temp_window_a23 = list(sheet['窗a23/℃'])
    return time,temp_door_d01,temp_door_d06,temp_door_d11,temp_door_d17,temp_door_d23,temp_door_c01,temp_door_c06,temp_door_c11,temp_door_c17,temp_door_c23,temp_door_b01,temp_door_b06,temp_door_b11,temp_door_b17,temp_door_b23,temp_door_a01,temp_door_a06,temp_door_a11,temp_door_a17,temp_door_a23,temp_mid_d01,temp_mid_d06,temp_mid_d11,temp_mid_d17,temp_mid_d23,temp_mid_c01,temp_mid_c06,temp_mid_c11,temp_mid_c17,temp_mid_c23,temp_mid_b01,temp_mid_b06,temp_mid_b11,temp_mid_b17,temp_mid_b23,temp_mid_a01,temp_mid_a06,temp_mid_a11,temp_mid_a17,temp_mid_a23,temp_window_d01,temp_window_d06,temp_window_d11,temp_window_d17,temp_window_d23,temp_window_c01,temp_window_c06,temp_window_c11,temp_window_c17,temp_window_c23,temp_window_b01,temp_window_b06,temp_window_b11,temp_window_b17,temp_window_b23,temp_window_a01,temp_window_a06,temp_window_a11,temp_window_a17,temp_window_a23

In [17]:
### [[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[ 定义参数 ]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]

##  --------------------------------------------- Part 1: 稳态时刻 房间平均参数-----------------------------------
    
#   稳态时刻供回水温度
list_temp_stable_suwat = []
list_temp_stable_rewat = []

#   稳态时刻末端表面温度
list_temp_stable_tersur = []   #平均温度
list_temp_stable_tersur1 = []  #测点1
list_temp_stable_tersur2 = []  #测点2
list_temp_stable_tersur3 = []  #测点3 
list_temp_stable_tersur4 = []  #测点4

#   室内平均空气温度, 室外平均温度
list_temp_stable_inair = []
list_temp_stable_outair = []

#   室外动态空气温度 （不需要输出）
list_temp_outair = []

#   非供暖表面温度
list_temp_stable_envelope = []  #平均温度
list_temp_stable_eastwall1 = [] #东墙1
list_temp_stable_eastwall2 = [] #东墙2
list_temp_stable_westwall1 = [] #西墙1
list_temp_stable_westwall2 = [] #西墙2
list_temp_stable_northwall = [] #北墙
list_temp_stable_southwall = [] #南墙
list_temp_stable_roof = []      #屋顶
list_temp_stable_window = []    #外窗
list_temp_stable_floor = []     #地面

#   操作温度
list_temp_stable_ope = []

#   稳态时刻热源供热量、末端表面供热量
list_heat_stable_suwat1 = []    #热源（用于末端自身特征）
list_heat_stable_suwat2 = []    #热源（用于末端对比）
list_heat_stable_tersur1 = []   #末端表面（用于末端自身特征）
list_heat_stable_tersur2 = []   #末端表面（用于末端对比）

#   ("末端表面稳定时间，单位h”)
#   ("室内空气温度时间，单位h”)
list_time_stable_tersur = []
list_time_stable_inair = []

#   最后1h室内空气温升
list_temprise_ope_lasthour = []

#   ("室内环境和末端表面总温差:" , tempdif_initial_final_inair , "和", tempdif_initial_final_tersur )
#   (不需要输出)
list_tempdif_inair = []
list_tempdif_tersur =[]

#   围护结构 (不需要输出)
list_temp_envelope = []

##  --------------------------------------------- Part2: 末端时间（传热）特征----------------------------------------

#   ("R1 is :",R1,"K/W","\nR2 is :",R2,"K/W", "\nC is :", C,"J/K")
#   等效传热热阻、等效散热热阻、等效热容
list_R1 =[]
list_R2 =[]
list_C =[]

## --------------------------------------------- Part4: 节能性 -----------------------------------------------------

#   热量-启动阶段：热源累积供热量、末端向室内累积散热量、末端累积蓄热量
list_cumheat_start_suwat = []
list_cumheat_start_tersur = []
list_cumheat_start_store = []

#   热量-稳定阶段：热源供热量、末端向室内散热量
#   在Part1中已定义

#   火积耗散-启动阶段：热源-末端累积火积耗散、末端-室内累积火积耗散、室内-室外累积火积耗散、总累积火积耗散
list_cumentransy_start_suwat_to_tersur = []
list_cumentransy_start_tersur_to_in = []
list_cumentransy_start_in_to_out = []
list_cumentransy_total = []

#   火积耗散-稳定阶段：热源-末端火积耗散、末端-室内火积耗散、室内-室外火积耗散、总火积耗散
list_entransy_stable_suwat_to_tersur = []
list_entransy_stable_tersur_to_in = []
list_entransy_stable_in_to_out = []
list_entransy_total = []

#   存储sheet1所需动态数据
sheet1_temp_suwat = []
sheet1_temp_tersur = []
sheet1_temp_outair = []
sheet1_temp_rewat = []
sheet1_volume_suwat = []

for sheet,path in list_data1:
    temp_suwat,temp_rewat,temp_inair,temp_tersur1,temp_tersur2,temp_tersur3,temp_tersur4,temp_outair,volume_suwat, \
    temp_eastwall1,temp_eastwall2,temp_westwall1,temp_westwall2,temp_northwall,temp_southwall,temp_roof,temp_window,temp_floor = read1(sheet)
    
### [[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[ 参数计算 ]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]

    sta_duration = 10  #单位，秒s
    temp_tersur = [(temp_tersur1[i]+temp_tersur3[i]+temp_tersur4[i])/3 for i in range(len(temp_tersur1))]  #暖气2号测点有问题，计算时去掉
    temp_envelope = [(temp_eastwall1[i]+temp_eastwall2[i]+temp_westwall1[i]+temp_westwall2[i]+temp_northwall[i]+temp_southwall[i]+temp_roof[i]+temp_floor[i])/8 for i in range(len(temp_floor))]

## ============================================== Part1：稳态时刻 房间平均参数 ===================================

#   粗略估计稳态时间（第六个小时，总运行时长的最后一小时）
    end_time = int(6*3600/sta_duration)
    start_time = int(end_time-3600/sta_duration)

#   .............................................稳态工况平均值：默认最后1h为稳态工况..............................

    temp_stable_suwat = ave(temp_suwat,start_time,end_time)
    temp_stable_rewat = ave(temp_rewat,start_time,end_time)
    temp_stable_inair = ave(temp_inair,start_time,end_time)
    temp_stable_tersur1 = ave(temp_tersur1,start_time,end_time)
    temp_stable_tersur2 = ave(temp_tersur2,start_time,end_time)
    temp_stable_tersur3 = ave(temp_tersur3,start_time,end_time)
    temp_stable_tersur4 = ave(temp_tersur4,start_time,end_time)
    temp_stable_tersur = ave(temp_tersur,start_time,end_time)
    temp_stable_outair = ave(temp_outair,0,end_time)
    temp_stable_eastwall1 = ave(temp_eastwall1,start_time,end_time)
    temp_stable_eastwall2 = ave(temp_eastwall2,start_time,end_time)
    temp_stable_westwall1 = ave(temp_westwall1,start_time,end_time)
    temp_stable_westwall2 = ave(temp_westwall2,start_time,end_time)
    temp_stable_northwall = ave(temp_northwall,start_time,end_time)
    temp_stable_southwall = ave(temp_southwall,start_time,end_time)
    temp_stable_roof = ave(temp_roof,start_time,end_time)
    temp_stable_window = ave(temp_window,start_time,end_time)
    temp_stable_floor = ave(temp_floor,start_time,end_time)
    temp_stable_envelope = ave(temp_envelope,start_time,end_time)

#   .............................................热量修正..........................................................

#   修正1：热量按供回水温差修正（由于回水温度测点有误）（根据供回水温差+1.2℃,现在表格里是加过1.2℃的，需要再减回去）
    error_wat = 1.2   #℃
    heat_suwat1 = repair_heat1(temp_suwat,temp_rewat,volume_suwat,error_wat)
    heat_suwat1_sta = ave(heat_suwat1,start_time,end_time)  #稳定时刻 

#   修正2：把热量归一化，即房间传热系数均为50W/K情况下
#   step1：修正系数（根据房间传热系数K=50W/K修正）
    print(heat_suwat1_sta)
    ita_env = 50/(heat_suwat1_sta/(temp_stable_inair - temp_stable_outair))
#   step2：修正后热源供热量
    heat_suwat2 = repair_heat2(heat_suwat1,ita_env)                      #逐时
    heat_suwat2_sta = ave(heat_suwat2,start_time,end_time)  #稳定时刻


#   ...................................计算末端表面和室内环境稳定时间（即达到始末温差95%）.........................

#   step1：始末温升
    tempdif_initial_final_inair = temp_stable_inair-temp_inair[0]         #室内空气
    tempdif_initial_final_tersur = temp_stable_tersur-temp_tersur[0]      #末端表面温度      
#   step2：计算逐时温度与初始温度之差
    tempdif_initial_t_inair = sub(temp_inair)     #室内空气
    tempdif_initial_t_tersur = sub(temp_tersur)   #末端表面温度
#   step3：逐时刻温升/最大温升，从这里面找到>95%的那个点，对应行数即为稳态时间（还需乘计数时间间隔）
    time_stable_inair = find_stable_time(tempdif_initial_t_inair,tempdif_initial_final_inair)       #室内空气
    time_stable_tersur = find_stable_time(tempdif_initial_t_tersur,tempdif_initial_final_tersur)    #末端表面温度

#   .............................................操作温度计算....................................................

    temp_ope = operative(temp_inair,temp_envelope)
    temp_ope_sta = ave (temp_ope,start_time,end_time)

#   .............................................给定义变量赋值....................................................

#   稳态时刻供回水温度
    list_temp_stable_suwat.append(temp_stable_suwat)
    list_temp_stable_rewat.append(temp_stable_rewat)
    
#   稳态时刻末端表面平均温度
    list_temp_stable_tersur.append(temp_stable_tersur)
    list_temp_stable_tersur1.append(temp_stable_tersur1)
    list_temp_stable_tersur2.append(temp_stable_tersur2)
    list_temp_stable_tersur3.append(temp_stable_tersur3)
    list_temp_stable_tersur4.append(temp_stable_tersur4)

#   稳态时刻室内平均空气温度, 室外平均温度
    list_temp_stable_inair.append(temp_stable_inair)
    list_temp_stable_outair.append(temp_stable_outair)

#   室外空气温度
    list_temp_outair.append(temp_outair)

#   稳态时刻非供暖表面平均温度
    list_temp_stable_envelope.append(temp_stable_envelope)
    list_temp_stable_eastwall1.append(temp_stable_eastwall1)
    list_temp_stable_eastwall2.append(temp_stable_eastwall2)
    list_temp_stable_westwall1.append(temp_stable_westwall1)
    list_temp_stable_westwall2.append(temp_stable_westwall2)
    list_temp_stable_northwall.append(temp_stable_northwall)
    list_temp_stable_southwall.append(temp_stable_southwall)
    list_temp_stable_roof.append(temp_stable_roof)
    list_temp_stable_window.append(temp_stable_window)
    list_temp_stable_floor.append(temp_stable_floor)   

#   稳态时刻供热量-热源
    list_heat_stable_suwat1.append(heat_suwat1_sta)          #修正后1（用于末端自身特征）
    list_heat_stable_suwat2.append(heat_suwat2_sta)          #修正后2（用于末端对比）
    
#   末端表面 和 室内空气 稳定时间：",stable_time_tersur*计数时间间隔s/3600，h)
    list_time_stable_tersur.append(time_stable_tersur*sta_duration/3600)
    list_time_stable_inair.append(time_stable_inair*sta_duration/3600)

    
#   稳态时刻操作温度
    list_temp_stable_ope.append(temp_ope_sta)
    
#   最后1h操作温度温升
    a = end_time
    b = int(end_time-3600/sta_duration)
    list_temprise_ope_lasthour.append(temp_ope[a]-temp_ope[b])
    
#   围护结构
    list_temp_envelope.append(temp_envelope)
    
##  =========================================== Part2: 末端时间（传热）特征 ========================================   
#   R1、R2、C（等效传热热阻、等效散热热阻、等效热容）
#   计算R1，R2，C时体现的是末端自身特征，需用末端[真实供热量]来计算

    #稳态工况（稳态时间，10）：R1 =ave【（供水温度-末端温度）/水侧供热量】【形成最后数组】
    #稳态工况（稳态时间，10）：R2 = ave【（末端温度-（室内空气温度+非供暖围护结构表面温度）/2）/水侧供热量】【形成最后数组】
    #启动工况（0，稳态时间）：C = sum（水侧供热量）* 60 /（稳态时间下末端温度-起始工况下末端温度）【形成最后数组】


    start_time = time_stable_inair

#   计算等效R1
    R1 = get_R1(temp_suwat,temp_tersur,heat_suwat1,start_time,end_time)

#   计算等效R2
    R2 = get_R2(temp_tersur,temp_ope,heat_suwat1,start_time,end_time)

#   计算等效C
    heat_tersur1 = get_Q(temp_tersur,temp_ope,R2,len(heat_suwat1))                   #计算末端向室内逐时供热量
    heat_store = [heat_suwat1[i]-heat_tersur1[i] for i in range(len(heat_suwat1))]  #计算末端逐时蓄热量
    tempdif_initial_stable_tersur = tempdif_initial_t_tersur[time_stable_tersur-1]   #稳定时刻末端表面温升=稳定温度-初始温度
    C = get_C(heat_store,tempdif_initial_stable_tersur,time_stable_tersur,sta_duration)  #计算C

#   .............................................给定义变量赋值....................................................

    print("R1 is :",R1,"K/W","\nR2 is :",R2,"K/W", "\nC is :", C,"J/K")
    list_R1.append(R1)
    list_R2.append(R2)
    list_C.append(C)

##  =========================================== Part4: 节能性 =======================================================

#   .............................................计算热量............................................................

#   启动阶段：热源累积供热量、末端向室内累积散热量、末端累积蓄热量
    heat_tersur2 = repair_heat2(heat_tersur1,ita_env)                         #用于末端对比（逐时）
    cumheat_start_suwat = sum(heat_suwat2[i] for i in range(0,time_stable_inair)) * sta_duration /3600/1000   #单位kWh
    cumheat_start_tersur = sum(heat_tersur2[i] for i in range(0,time_stable_inair)) * sta_duration /3600/1000  #单位kWh
    cumheat_start_store = sum(heat_store[i] for i in range(0,time_stable_inair)) * sta_duration /3600/1000    #单位kWh
    
#   稳定阶段：热源平均供热量（在part1中计算）、末端向室内平均散热量
    heat_tersur1_sta = ave(heat_tersur1,start_time,end_time)  #用于末端自身特征

    heat_tersur2_sta = ave(heat_tersur2,start_time,end_time)  #用于末端对比

    
#   ...........................计算火积耗散（假设散热器60%热量为辐射，40%为对流）....................................

#   逐时火积耗散（动态）
    entransy_heattransfer_source_to_medium = [(temp_suwat[i]-temp_rewat[i])/2*heat_suwat2[i] for i in range(len(heat_suwat2))]
    entransy_heatstorage_medium_to_tersur = [((temp_suwat[i]+temp_rewat[i])/2-temp_tersur[i])*(heat_suwat2[i]-heat_tersur2[i]) for i in range(len(heat_suwat2))]
    entransy_heattransfer_medium_to_tersur = [((temp_suwat[i]+temp_rewat[i])/2-temp_tersur[i])*heat_tersur2[i] for i in range(len(heat_tersur2))]
    entransy_heattransfer_tersur_to_inair = [(temp_tersur[i]-temp_inair[i])*heat_tersur2[i]*0.4 for i in range(len(heat_tersur2))]
    entransy_heattransfer_tersur_to_envelope = [(temp_tersur[i]-temp_envelope[i])*heat_tersur2[i]*0.6 for i in range(len(heat_tersur2))]
    entransy_heattransfer_inair_to_outair = [(temp_inair[i]-temp_outair[i])*heat_tersur2[i]*0.4 for i in range(len(heat_tersur2))]
    entransy_heattransfer_envelope_to_outair = [(temp_envelope[i]-temp_outair[i])*heat_tersur2[i]*0.6 for i in range(len(heat_tersur2))]

#   启动阶段:热源-末端累积火积耗散、末端-室内累积火积耗散、室内-室外累积火积耗散、总累积火积耗散
    cumentransy_start_suwat_to_tersur = (sum(entransy_heattransfer_source_to_medium[i] for i in range(0,time_stable_inair)) + sum(entransy_heatstorage_medium_to_tersur[i] for i in range(0,time_stable_inair)) + sum(entransy_heattransfer_medium_to_tersur[i] for i in range(0,time_stable_inair))) * sta_duration /3600/1000
    cumentransy_start_tersur_to_in = (sum(entransy_heattransfer_tersur_to_envelope[i] for i in range(0,time_stable_inair)) + sum(entransy_heattransfer_tersur_to_inair[i] for i in range(0,time_stable_inair))) * sta_duration /3600/1000
    cumentransy_start_in_to_out = (sum(entransy_heattransfer_inair_to_outair [i] for i in range(0,time_stable_inair)) + sum(entransy_heattransfer_envelope_to_outair[i] for i in range(0,time_stable_inair))) * sta_duration /3600/1000
    cumentransy_total = cumentransy_start_suwat_to_tersur + cumentransy_start_tersur_to_in + cumentransy_start_in_to_out
    
#   稳定阶段：热源-末端火积耗散、末端-室内火积耗散、室内-室外火积耗散、总火积耗散    
    entransy_stable_suwat_to_tersur = ave(entransy_heattransfer_source_to_medium,time_stable_inair,end_time)+ ave(entransy_heatstorage_medium_to_tersur,time_stable_inair,end_time)+ ave(entransy_heattransfer_medium_to_tersur,time_stable_inair,end_time)  #单位，WK
    entransy_stable_tersur_to_in = ave(entransy_heattransfer_tersur_to_inair,time_stable_inair,end_time) + ave(entransy_heattransfer_tersur_to_envelope,time_stable_inair,end_time)
    entransy_stable_in_to_out = ave(entransy_heattransfer_inair_to_outair,time_stable_inair,end_time) + ave(entransy_heattransfer_envelope_to_outair,time_stable_inair,end_time)
    entransy_total = entransy_stable_suwat_to_tersur + entransy_stable_tersur_to_in + entransy_stable_in_to_out

#   .............................................给定义变量赋值....................................................

#   热量-启动阶段：热源累积供热量、末端向室内累积散热量、末端累积蓄热量
    list_cumheat_start_suwat.append(cumheat_start_suwat)
    list_cumheat_start_tersur.append(cumheat_start_tersur)
    list_cumheat_start_store.append(cumheat_start_store)

#   热量-稳定阶段：末端向室内散热量
    list_heat_stable_tersur1.append(heat_tersur1_sta)   #末端表面（用于末端自身特征）
    list_heat_stable_tersur2.append(heat_tersur2_sta)   #末端表面（用于末端对比）

#   火积耗散-启动阶段：热源-末端累积火积耗散、末端-室内累积火积耗散、室内-室外累积火积耗散、总累积火积耗散
    list_cumentransy_start_suwat_to_tersur.append(cumentransy_start_suwat_to_tersur)
    list_cumentransy_start_tersur_to_in.append(cumentransy_start_tersur_to_in)
    list_cumentransy_start_in_to_out.append(cumentransy_start_in_to_out)
    list_cumentransy_total.append(cumentransy_total)

#   火积耗散-稳定阶段：热源-末端火积耗散、末端-室内火积耗散、室内-室外火积耗散、总火积耗散
    list_entransy_stable_suwat_to_tersur.append(entransy_stable_suwat_to_tersur)
    list_entransy_stable_tersur_to_in.append(entransy_stable_tersur_to_in)
    list_entransy_stable_in_to_out.append(entransy_stable_in_to_out)
    list_entransy_total.append(entransy_total)

##  =========================================== 获取动态数据 =======================================================

    sheet1_temp_suwat.append(temp_suwat)
    sheet1_temp_tersur.append(temp_tersur)
    sheet1_temp_outair.append(temp_outair)
    sheet1_temp_rewat.append(temp_rewat)
    sheet1_volume_suwat.append(volume_suwat)
    

1199.3272809719288
R1 is : 0.0031253096624022886 K/W 
R2 is : 0.030108387485243384 K/W 
C is : 89518.07175101749 J/K
1150.6106371716758
R1 is : 0.003146784982838487 K/W 
R2 is : 0.03232637768672838 K/W 
C is : 91224.83191075096 J/K
1267.37809798685
R1 is : 0.0033698391190877517 K/W 
R2 is : 0.030648681534933796 K/W 
C is : 83577.76935298042 J/K
1184.539336927748
R1 is : 0.004258832076382196 K/W 
R2 is : 0.02740014536126225 K/W 
C is : 134709.8134308482 J/K


In [18]:
##  --------------------------------------------- Part3: 末端空间特征 ------------------------------------------------

#   启动时刻：zoneA温度、zoneB温度
list_temp_initial_zoneA = []
list_temp_initial_zoneB = []
list_temp_initial_zoneC = []
list_temp_initial_zoneD = []

#   稳态时刻：zoneA平均温度、zoneB平均温度、1.7-0.1垂直温差、1.1-0.1垂直温差
list_temp_stable_zoneA = []
list_temp_stable_zoneB = []
list_temp_stable_zoneC = []
list_temp_stable_zoneD = []
list_tempdif_stable_17to01 = []
list_tempdif_stable_11to01 = []

#   热量匹配系数HMC（用温升算）：zoneA区0.5h热量匹配系数、zoneB区0.5h热量匹配系数、zoneA区启动阶段热量匹配系数、zoneB区启动阶段热量匹配系数
list_HMC1_zoneA_30min = []
list_HMC1_zoneB_30min = []
list_HMC1_zoneC_30min = []
list_HMC1_zoneD_30min = []
list_HMC1_zoneA_start = []
list_HMC1_zoneB_start = []
list_HMC1_zoneC_start = []
list_HMC1_zoneD_start = []

#   热量匹配系数HMC（用室内外温差算）：zoneA区启动阶段热量匹配系数、zoneB区启动阶段热量匹配系数、zoneA区稳态阶段热量匹配系数、zongB区稳态阶段热量匹配系数
list_HMC2_zoneA_start = []
list_HMC2_zoneB_start = []
list_HMC2_zoneA_stable = []
list_HMC2_zoneB_stable = []

## --------------------------------------------- Part5: 舒适性 -----------------------------------------------------

#   时间：zoneA上升10℃所用时间，zoneB上升10℃所用时间，室内空气温度上升10℃所用时间
list_time_zoneA = []
list_time_zoneB = []
list_time_total =[]

#   0.5h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
list_TMC_zoneA_30min = []
list_TMC_zoneB_30min = []
list_TMC_total_30min = []

#   1h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
list_TMC_zoneA_1h = []
list_TMC_zoneB_1h = []
list_TMC_total_1h = []

#   2h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
list_TMC_zoneA_2h = []
list_TMC_zoneB_2h = []
list_TMC_total_2h = []

#   3h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
list_TMC_zoneA_3h = []
list_TMC_zoneB_3h = []
list_TMC_total_3h = []

#   4h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
list_TMC_zoneA_4h = []
list_TMC_zoneB_4h = []
list_TMC_total_4h = []

#   5h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
list_TMC_zoneA_5h = []
list_TMC_zoneB_5h = []
list_TMC_total_5h = []

#   6h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
list_TMC_zoneA_6h = []
list_TMC_zoneB_6h = []
list_TMC_total_6h = []

list_time_initial_to_63_zoneA = []
list_time_initial_to_63_zoneB = []
list_time_initial_to_63_total = []
list_HMC1_zoneA_63 = []
list_HMC1_zoneB_63 = []


list_W_HMC63 = []
list_W_HMC95 = []
list_W_heat_tersur = []

id_sheet1 = 0 # index
for sheet,path in list_data2:
    time,temp_door_d01,temp_door_d06,temp_door_d11,temp_door_d17,temp_door_d23,temp_door_c01,temp_door_c06,temp_door_c11,temp_door_c17,temp_door_c23,temp_door_b01,temp_door_b06,temp_door_b11,temp_door_b17,temp_door_b23,temp_door_a01,temp_door_a06,temp_door_a11,temp_door_a17,temp_door_a23, \
    temp_mid_d01,temp_mid_d06,temp_mid_d11,temp_mid_d17,temp_mid_d23,temp_mid_c01,temp_mid_c06,temp_mid_c11,temp_mid_c17,temp_mid_c23,temp_mid_b01,temp_mid_b06,temp_mid_b11,temp_mid_b17,temp_mid_b23,temp_mid_a01,temp_mid_a06,temp_mid_a11,temp_mid_a17,temp_mid_a23, \
    temp_window_d01,temp_window_d06,temp_window_d11,temp_window_d17,temp_window_d23,temp_window_c01,temp_window_c06,temp_window_c11,temp_window_c17,temp_window_c23,temp_window_b01,temp_window_b06,temp_window_b11,temp_window_b17,temp_window_b23,temp_window_a01,temp_window_a06,temp_window_a11,temp_window_a17,temp_window_a23 = read2(sheet)
    
### [[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[ 参数计算 ]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]

## 读取sheet1的数值
    time_stable_tersur = list_time_stable_tersur[id_sheet1]
    temp_outair = list_temp_outair[id_sheet1]
    temp_stable_outair = list_temp_stable_outair[id_sheet1]
    temp_envelope = list_temp_envelope[id_sheet1]
    temp_suwat = sheet1_temp_suwat[id_sheet1]
    temp_rewat = sheet1_temp_rewat[id_sheet1]
    volume_suwat = sheet1_volume_suwat[id_sheet1]
    
    id_sheet1 = id_sheet1 + 1
    

## ------------------------------------------------ Part3：末端空间特征--------------------------------------------

    H_zoneA = 1.3   #下层A区域高度，m
    H_zoneB = 1.3   #上层B区域高度，m

    temp_01m = [(temp_door_d01[i]+temp_door_c01[i]+temp_door_b01[i]+temp_door_a01[i]+temp_mid_d01[i]+temp_mid_c01[i]+temp_mid_b01[i]+temp_mid_a01[i]+temp_window_d01[i]+temp_window_c01[i]+temp_window_b01[i]+temp_window_a01[i])/12 for i in range(len(temp_door_d01))]
    temp_06m = [(temp_door_d06[i]+temp_door_c06[i]+temp_door_b06[i]+temp_door_a06[i]+temp_mid_d06[i]+temp_mid_c06[i]+temp_mid_b06[i]+temp_mid_a06[i]+temp_window_d06[i]+temp_window_c06[i]+temp_window_b06[i]+temp_window_a06[i])/12 for i in range(len(temp_door_d06))]
    temp_11m = [(temp_door_d11[i]+temp_door_c11[i]+temp_door_b11[i]+temp_door_a11[i]+temp_mid_d11[i]+temp_mid_c11[i]+temp_mid_b11[i]+temp_mid_a11[i]+temp_window_d11[i]+temp_window_c11[i]+temp_window_b11[i]+temp_window_a11[i])/12 for i in range(len(temp_door_d11))]
    temp_17m = [(temp_door_d17[i]+temp_door_c17[i]+temp_door_b17[i]+temp_door_a17[i]+temp_mid_d17[i]+temp_mid_c17[i]+temp_mid_b17[i]+temp_mid_a17[i]+temp_window_d17[i]+temp_window_c17[i]+temp_window_b17[i]+temp_window_a17[i])/12 for i in range(len(temp_door_d17))]
    temp_23m = [(temp_door_d23[i]+temp_door_c23[i]+temp_door_b23[i]+temp_door_a23[i]+temp_mid_d23[i]+temp_mid_c23[i]+temp_mid_b23[i]+temp_mid_a23[i]+temp_window_d23[i]+temp_window_c23[i]+temp_window_b23[i]+temp_window_a23[i])/12 for i in range(len(temp_door_d23))]

    temp_01m_C = [(temp_door_b01[i]+temp_door_a01[i]+temp_mid_b01[i]+temp_mid_a01[i]+temp_window_b01[i]+temp_window_a01[i])/6 for i in range(len(temp_door_a01))]
    temp_06m_C = [(temp_door_b06[i]+temp_door_a06[i]+temp_mid_b06[i]+temp_mid_a06[i]+temp_window_b06[i]+temp_window_a06[i])/6 for i in range(len(temp_door_a06))]
    temp_11m_C = [(temp_door_b11[i]+temp_door_a11[i]+temp_mid_b11[i]+temp_mid_a11[i]+temp_window_b11[i]+temp_window_a11[i])/6 for i in range(len(temp_door_a11))]    
   
    temp_01m_D = [(temp_door_d01[i]+temp_door_c01[i]+temp_mid_d01[i]+temp_mid_c01[i]+temp_window_d01[i]+temp_window_c01[i])/6 for i in range(len(temp_door_d01))]
    temp_06m_D = [(temp_door_d06[i]+temp_door_c06[i]+temp_mid_d06[i]+temp_mid_c06[i]+temp_window_d06[i]+temp_window_c06[i])/6 for i in range(len(temp_door_d06))]
    temp_11m_D = [(temp_door_d11[i]+temp_door_c11[i]+temp_mid_d11[i]+temp_mid_c11[i]+temp_window_d11[i]+temp_window_c11[i])/6 for i in range(len(temp_door_d11))]    

    temp_02m = [temp_01m[i]+(temp_06m[i]-temp_01m[i])/5*1 for i in range(len(temp_01m))]
    temp_03m = [temp_01m[i]+(temp_06m[i]-temp_01m[i])/5*2 for i in range(len(temp_01m))]
    temp_04m = [temp_01m[i]+(temp_06m[i]-temp_01m[i])/5*3 for i in range(len(temp_01m))]
    temp_05m = [temp_01m[i]+(temp_06m[i]-temp_01m[i])/5*4 for i in range(len(temp_01m))]
    
    temp_07m = [temp_06m[i]+(temp_11m[i]-temp_06m[i])/5*1 for i in range(len(temp_06m))]
    temp_08m = [temp_06m[i]+(temp_11m[i]-temp_06m[i])/5*2 for i in range(len(temp_06m))]
    temp_09m = [temp_06m[i]+(temp_11m[i]-temp_06m[i])/5*3 for i in range(len(temp_06m))]
    temp_10m = [temp_06m[i]+(temp_11m[i]-temp_06m[i])/5*4 for i in range(len(temp_06m))]
    
    temp_12m = [temp_11m[i]+(temp_17m[i]-temp_11m[i])/6*1 for i in range(len(temp_11m))]
    temp_13m = [temp_11m[i]+(temp_17m[i]-temp_11m[i])/6*2 for i in range(len(temp_11m))]
    temp_14m = [temp_11m[i]+(temp_17m[i]-temp_11m[i])/6*3 for i in range(len(temp_11m))]
    temp_15m = [temp_11m[i]+(temp_17m[i]-temp_11m[i])/6*4 for i in range(len(temp_11m))]
    temp_16m = [temp_11m[i]+(temp_17m[i]-temp_11m[i])/6*5 for i in range(len(temp_11m))]
    
    temp_18m = [temp_17m[i]+(temp_23m[i]-temp_17m[i])/6*1 for i in range(len(temp_17m))]
    temp_19m = [temp_17m[i]+(temp_23m[i]-temp_17m[i])/6*2 for i in range(len(temp_17m))]
    temp_20m = [temp_17m[i]+(temp_23m[i]-temp_17m[i])/6*3 for i in range(len(temp_17m))]
    temp_21m = [temp_17m[i]+(temp_23m[i]-temp_17m[i])/6*4 for i in range(len(temp_17m))]
    temp_22m = [temp_17m[i]+(temp_23m[i]-temp_17m[i])/6*5 for i in range(len(temp_17m))]
    
    temp_24m = [temp_17m[i]+(temp_23m[i]-temp_17m[i])/6*7 for i in range(len(temp_17m))]
    temp_25m = [temp_17m[i]+(temp_23m[i]-temp_17m[i])/6*8 for i in range(len(temp_17m))]
    
    temp_zoneA = [(temp_01m[i]+temp_02m[i]+temp_03m[i]+temp_04m[i]+temp_05m[i]+temp_06m[i]+temp_07m[i]+temp_08m[i]+temp_09m[i]+temp_10m[i]+temp_11m[i]+temp_12m[i]+temp_13m[i])/13 for i in range(len(temp_01m))]
    temp_zoneB = [(temp_13m[i]+temp_14m[i]+temp_15m[i]+temp_16m[i]+temp_17m[i]+temp_18m[i]+temp_19m[i]+temp_20m[i]+temp_21m[i]+temp_22m[i]+temp_23m[i]+temp_24m[i]+temp_25m[i])/13 for i in range(len(temp_17m))]
    temp_total = [(temp_zoneA[i]*H_zoneA+temp_zoneB[i]*H_zoneB)/(H_zoneA+H_zoneB) for i in range(len(temp_01m))]
    temp_zoneC = [(temp_01m_C[i]*0.35+temp_06m_C[i]*0.5+temp_11m_C[i]*0.45)/H_zoneA for i in range(len(temp_01m))]
    temp_zoneD = [(temp_01m_D[i]*0.35+temp_06m_D[i]*0.5+temp_11m_D[i]*0.45)/H_zoneA for i in range(len(temp_01m))]
    
#   启动时刻：zoneA温度、zoneB温度
    temp_initial_zoneA = temp_zoneA[0]
    temp_initial_zoneB = temp_zoneB[0]
    temp_initial_zoneC = temp_zoneC[0]
    temp_initial_zoneD = temp_zoneD[0]
    temp_initial_total = (temp_initial_zoneA + temp_initial_zoneB) / 2

#   稳态时刻：zoneA平均温度、zoneB平均温度、1.7-0.1垂直温差、1.1-0.1垂直温差

    temp_stable_zoneA = ave(temp_zoneA,start_time,end_time)
    temp_stable_zoneB = ave(temp_zoneB,start_time,end_time)
    temp_stable_zoneC = ave(temp_zoneC,start_time,end_time)
    temp_stable_zoneD = ave(temp_zoneD,start_time,end_time)
    temp_stable_total = (temp_stable_zoneA + temp_stable_zoneB) / 2
    tempdif_stable_17to01 = ave(temp_17m,start_time,end_time)-ave(temp_01m,start_time,end_time)
    tempdif_stable_11to01 = ave(temp_11m,start_time,end_time)-ave(temp_01m,start_time,end_time)
    
#   ..................................................热量匹配系数HMC..............................................

#   用温升算：zoneA区0.5h热量匹配系数、zoneB区0.5h热量匹配系数、zoneA区启动阶段热量匹配系数、zoneB区启动阶段热量匹配系数

    temp_up_total_01m = temp_up_total(temp_01m)     #01m累积温升：逐时刻温度-初始温度
    temp_up_total_02m = temp_up_total(temp_02m)
    temp_up_total_03m = temp_up_total(temp_03m)
    temp_up_total_04m = temp_up_total(temp_04m)
    temp_up_total_05m = temp_up_total(temp_05m)
    temp_up_total_06m = temp_up_total(temp_06m)     #06m累积温升：逐时刻温度-初始温度
    temp_up_total_07m = temp_up_total(temp_07m)
    temp_up_total_08m = temp_up_total(temp_08m)
    temp_up_total_09m = temp_up_total(temp_09m)
    temp_up_total_10m = temp_up_total(temp_10m)
    temp_up_total_11m = temp_up_total(temp_11m)     #11m累积温升：逐时刻温度-初始温度
    temp_up_total_12m = temp_up_total(temp_12m)
    temp_up_total_13m = temp_up_total(temp_13m)
    temp_up_total_14m = temp_up_total(temp_14m)
    temp_up_total_15m = temp_up_total(temp_15m)
    temp_up_total_16m = temp_up_total(temp_16m)
    temp_up_total_17m = temp_up_total(temp_17m)     #17m累积温升：逐时刻温度-初始温度
    temp_up_total_18m = temp_up_total(temp_18m)
    temp_up_total_19m = temp_up_total(temp_19m)
    temp_up_total_20m = temp_up_total(temp_20m)
    temp_up_total_21m = temp_up_total(temp_21m)
    temp_up_total_22m = temp_up_total(temp_22m)
    temp_up_total_23m = temp_up_total(temp_23m)     #23m累积温升：逐时刻温度-初始温度
    temp_up_total_24m = temp_up_total(temp_24m)
    temp_up_total_25m = temp_up_total(temp_25m)
    
    temp_up_total_01m_C = temp_up_total(temp_01m_C)     #01m累积温升：逐时刻温度-初始温度
    temp_up_total_06m_C = temp_up_total(temp_06m_C)     #06m累积温升：逐时刻温度-初始温度
    temp_up_total_11m_C = temp_up_total(temp_11m_C)     #11m累积温升：逐时刻温度-初始温度   
    temp_up_total_01m_D = temp_up_total(temp_01m_D)     #01m累积温升：逐时刻温度-初始温度
    temp_up_total_06m_D = temp_up_total(temp_06m_D)     #06m累积温升：逐时刻温度-初始温度
    temp_up_total_11m_D = temp_up_total(temp_11m_D)     #11m累积温升：逐时刻温度-初始温度  
    
    heat1_zoneA = [(temp_up_total_01m[i]+temp_up_total_02m[i]+temp_up_total_03m[i]+temp_up_total_04m[i]+temp_up_total_05m[i]+temp_up_total_06m[i]+temp_up_total_07m[i]+temp_up_total_08m[i]+temp_up_total_09m[i]+temp_up_total_10m[i]+temp_up_total_11m[i]+temp_up_total_12m[i]+temp_up_total_13m[i])*0.1 for i in range(len(temp_up_total_11m))]
    heat1_zoneB = [(temp_up_total_13m[i]+temp_up_total_14m[i]+temp_up_total_15m[i]+temp_up_total_16m[i]+temp_up_total_17m[i]+temp_up_total_18m[i]+temp_up_total_19m[i]+temp_up_total_20m[i]+temp_up_total_21m[i]+temp_up_total_22m[i]+temp_up_total_23m[i]+temp_up_total_24m[i]+temp_up_total_25m[i])*0.1 for i in range(len(temp_up_total_17m))]
    heat1_total = [heat1_zoneA[i]+heat1_zoneB[i] for i in range(len(heat1_zoneA))]
    heat1_zoneC = [(temp_up_total_01m_C[i]*0.35+temp_up_total_06m_C[i]*0.5+temp_up_total_11m_C[i]*0.45)/2 for i in range(len(temp_up_total_11m_C))]
    heat1_zoneD = [(temp_up_total_01m_D[i]*0.35+temp_up_total_06m_D[i]*0.5+temp_up_total_11m_D[i]*0.45)/2 for i in range(len(temp_up_total_11m_D))]   
    
    # 问题： 怀疑注释掉的部分的heat1_intotal写错了，应该是heat1_total。如果确实如此，可以直接将注释部分删掉。
    HMC1_zoneA_dynamic = [heat1_zoneA[i] / heat1_total[i] for i in range(0,end_time-1)]   #A区域动态热量匹配系数
    HMC1_zoneB_dynamic = [heat1_zoneB[i] / heat1_total[i] for i in range(0,end_time-1)]   #B区域动态热量匹配系数
    HMC1_zoneC_dynamic = [heat1_zoneC[i] / heat1_total[i] for i in range(0,end_time-1)]   #C区域动态热量匹配系数
    HMC1_zoneD_dynamic = [heat1_zoneD[i] / heat1_total[i] for i in range(0,end_time-1)]   #D区域动态热量匹配系数
    
    # 问题： 我修改了代码，//是int的除法，/是float的除法。
    HMC1_zoneA_30min = HMC1_zoneA_dynamic[(30*60)//10]    #zoneA区0.5h热量匹配系数
    HMC1_zoneB_30min = HMC1_zoneB_dynamic[(30*60)//10]    #zoneB区0.5h热量匹配系数
    HMC1_zoneC_30min = HMC1_zoneC_dynamic[(30*60)//10]    #zoneC区0.5h热量匹配系数
    HMC1_zoneD_30min = HMC1_zoneD_dynamic[(30*60)//10]    #zoneD区0.5h热量匹配系数
    
    # 问题： 由于time_stable_tersur并不是整数，所以计算会有问题。我擅自修改了代码，意思是取“下整数”
    # 而且，我用end_time-1代替了end_time
    HMC1_zoneA_start = np.mean([HMC1_zoneA_dynamic[i] for i in range(int(np.floor(time_stable_tersur*3600/10)),end_time-1)])  #zoneA区启动阶段热量匹配系数
    HMC1_zoneB_start = np.mean([HMC1_zoneB_dynamic[i] for i in range(int(np.floor(time_stable_tersur*3600/10)),end_time-1)])  #zoneB区启动阶段热量匹配系数
    HMC1_zoneC_start = np.mean([HMC1_zoneC_dynamic[i] for i in range(int(np.floor(time_stable_tersur*3600/10)),end_time-1)])  #zoneC区启动阶段热量匹配系数
    HMC1_zoneD_start = np.mean([HMC1_zoneD_dynamic[i] for i in range(int(np.floor(time_stable_tersur*3600/10)),end_time-1)])  #zoneD区启动阶段热量匹配系数
    

#   用室内外温差算：zoneA区启动阶段热量匹配系数、zoneB区启动阶段热量匹配系数、zoneA区稳态阶段热量匹配系数、zongB区稳态阶段热量匹配系数
    
    tempdif_out_01m = dynamic_tempdif(temp_01m,temp_outair)
    tempdif_out_02m = dynamic_tempdif(temp_02m,temp_outair)
    tempdif_out_03m = dynamic_tempdif(temp_03m,temp_outair)
    tempdif_out_04m = dynamic_tempdif(temp_04m,temp_outair)
    tempdif_out_05m = dynamic_tempdif(temp_05m,temp_outair)
    tempdif_out_06m = dynamic_tempdif(temp_06m,temp_outair)
    tempdif_out_07m = dynamic_tempdif(temp_07m,temp_outair)
    tempdif_out_08m = dynamic_tempdif(temp_08m,temp_outair)
    tempdif_out_09m = dynamic_tempdif(temp_09m,temp_outair)
    tempdif_out_10m = dynamic_tempdif(temp_10m,temp_outair)
    tempdif_out_11m = dynamic_tempdif(temp_11m,temp_outair)
    tempdif_out_12m = dynamic_tempdif(temp_12m,temp_outair)
    tempdif_out_13m = dynamic_tempdif(temp_13m,temp_outair)
    tempdif_out_14m = dynamic_tempdif(temp_14m,temp_outair)
    tempdif_out_15m = dynamic_tempdif(temp_15m,temp_outair)
    tempdif_out_16m = dynamic_tempdif(temp_16m,temp_outair)
    tempdif_out_17m = dynamic_tempdif(temp_17m,temp_outair)
    tempdif_out_18m = dynamic_tempdif(temp_18m,temp_outair)
    tempdif_out_19m = dynamic_tempdif(temp_19m,temp_outair)
    tempdif_out_20m = dynamic_tempdif(temp_20m,temp_outair)
    tempdif_out_21m = dynamic_tempdif(temp_21m,temp_outair)
    tempdif_out_22m = dynamic_tempdif(temp_22m,temp_outair)
    tempdif_out_23m = dynamic_tempdif(temp_23m,temp_outair)
    tempdif_out_24m = dynamic_tempdif(temp_24m,temp_outair)
    tempdif_out_25m = dynamic_tempdif(temp_25m,temp_outair)
    
    heat2_zoneA = [(tempdif_out_01m[i]+tempdif_out_02m[i]+tempdif_out_03m[i]+tempdif_out_04m[i]+tempdif_out_05m[i]+tempdif_out_06m[i]+tempdif_out_07m[i]+tempdif_out_08m[i]+tempdif_out_09m[i]+tempdif_out_10m[i]+tempdif_out_11m[i]+tempdif_out_12m[i]+tempdif_out_13m[i])*0.1 for i in range(len(tempdif_out_11m))]
    heat2_zoneB = [(tempdif_out_13m[i]+tempdif_out_14m[i]+tempdif_out_15m[i]+tempdif_out_16m[i]+tempdif_out_17m[i]+tempdif_out_18m[i]+tempdif_out_19m[i]+tempdif_out_20m[i]+tempdif_out_21m[i]+tempdif_out_22m[i]+tempdif_out_23m[i]+tempdif_out_24m[i]+tempdif_out_25m[i])*0.1 for i in range(len(tempdif_out_17m))]
    heat2_total = [heat2_zoneA[i] + heat2_zoneB[i] for i in range(len(heat2_zoneA))]
    
    
    # 问题： 怀疑注释掉的部分的heat2_intotal写错了，应该是heat2_total。如果确实如此，可以直接将注释部分删掉。
    HMC2_zoneA_dynamic = [heat2_zoneA[i] / heat2_total[i] for i in range(0,end_time-1)]   #A区域动态热量匹配系数
    HMC2_zoneB_dynamic = [heat2_zoneB[i] / heat2_total[i] for i in range(0,end_time-1)]   #B区域动态热量匹配系数
    
    # 类似问题： 由于time_stable_tersur并不是整数，所以计算会有问题。我擅自修改了代码，意思是取“下整数”
    # 而且，我用end_time-1代替了end_time
    HMC2_zoneA_start = np.mean([HMC2_zoneA_dynamic[i] for i in range(0,int(np.floor(time_stable_tersur*3600/10)))])  #zoneA区启动阶段热量匹配系数
    HMC2_zoneB_start = np.mean([HMC2_zoneB_dynamic[i] for i in range(0,int(np.floor(time_stable_tersur*3600/10)))])  #zoneB区启动阶段热量匹配系数
    
    HMC2_zoneA_stable = np.mean([HMC2_zoneA_dynamic[i] for i in range(int(np.floor(time_stable_tersur*3600/10)),end_time-1)])  #zoneA区稳态阶段热量匹配系数
    HMC2_zoneB_stable = np.mean([HMC2_zoneB_dynamic[i] for i in range(int(np.floor(time_stable_tersur*3600/10)),end_time-1)])  #zoneB区稳态阶段热量匹配系数    

#   .............................................给定义变量赋值....................................................
       
#   启动时刻：zoneA温度、zoneB温度
    list_temp_initial_zoneA.append(temp_initial_zoneA)
    list_temp_initial_zoneB.append(temp_initial_zoneB)
    list_temp_initial_zoneC.append(temp_initial_zoneC)
    list_temp_initial_zoneD.append(temp_initial_zoneD)
    

#   稳态时刻：zoneA平均温度、zoneB平均温度、1.7-0.1垂直温差、1.1-0.1垂直温差
    list_temp_stable_zoneA.append(temp_stable_zoneA)
    list_temp_stable_zoneB.append(temp_stable_zoneB)
    list_temp_stable_zoneC.append(temp_stable_zoneC)
    list_temp_stable_zoneD.append(temp_stable_zoneD)    
    list_tempdif_stable_17to01.append(tempdif_stable_17to01)
    list_tempdif_stable_11to01.append(tempdif_stable_11to01)

#   热量匹配系数HMC（用温升算）：zoneA区0.5h热量匹配系数、zoneB区0.5h热量匹配系数、zoneA区启动阶段热量匹配系数、zoneB区启动阶段热量匹配系数
    list_HMC1_zoneA_30min.append(HMC1_zoneA_30min)
    list_HMC1_zoneB_30min.append(HMC1_zoneB_30min)
    list_HMC1_zoneA_start.append(HMC1_zoneA_start)
    list_HMC1_zoneB_start.append(HMC1_zoneB_start)
    list_HMC1_zoneC_30min.append(HMC1_zoneC_30min)
    list_HMC1_zoneD_30min.append(HMC1_zoneD_30min)
    list_HMC1_zoneC_start.append(HMC1_zoneC_start)
    list_HMC1_zoneD_start.append(HMC1_zoneD_start)

#   热量匹配系数HMC（用室内外温差算）：zoneA区启动阶段热量匹配系数、zoneB区启动阶段热量匹配系数、zoneA区稳态阶段热量匹配系数、zongB区稳态阶段热量匹配系数
    list_HMC2_zoneA_start.append(HMC2_zoneA_start)
    list_HMC2_zoneB_start.append(HMC2_zoneB_start)
    list_HMC2_zoneA_stable.append(HMC2_zoneA_stable)
    list_HMC2_zoneB_stable.append(HMC2_zoneB_stable)

## ---------------------------------------------------- Part5: 舒适性 -----------------------------------------------------    
    
    temp_target = temp_stable_outair + 15
#     temp_operative_target = temp_stable_outair + 15
#     temp_operative_zoneA = operative(temp_zoneA,temp_envelope)
#     temp_operative_zoneB = operative(temp_zoneB,temp_envelope)
#     temp_operative_total = operative(temp_total,temp_envelope)
#     temp_operative_zoneC = operative(temp_zoneC,temp_envelope)
#     temp_operative_zoneD = operative(temp_zoneD,temp_envelope)
    
    operating_duration_30min = int(0.5*3600)//10
    operating_duration_1h = 1*3600//10
    operating_duration_2h = 2*3600//10
    operating_duration_3h = 3*3600//10
    operating_duration_4h = 4*3600//10
    operating_duration_5h = 5*3600//10
    operating_duration_6h = 6*3600//10

    time_initial_to_target_zoneA = time_to_arrive(temp_zoneA,temp_target-15*0.37)*sta_duration/3600
    time_initial_to_target_zoneB = time_to_arrive(temp_zoneB,temp_target-15*0.37)*sta_duration/3600    
    time_initial_to_target_total = time_to_arrive(temp_total,temp_target-15*0.37)*sta_duration/3600
#     time_initial_to_target_zoneA = time_to_arrive(temp_operative_zoneA,temp_operative_target-2)*sta_duration/3600
#     time_initial_to_target_zoneB = time_to_arrive(temp_operative_zoneB,temp_operative_target-2)*sta_duration/3600    
#     time_initial_to_target_total = time_to_arrive(temp_operative_total,temp_operative_target-2)*sta_duration/3600
    time_initial_to_63_zoneA = time_to_arrive(temp_zoneA,temp_initial_zoneA+(temp_stable_zoneA-temp_initial_zoneA)*0.63)
    time_initial_to_63_zoneB = time_to_arrive(temp_zoneB,temp_initial_zoneB+(temp_stable_zoneB-temp_initial_zoneB)*0.63)
    time_initial_to_63_total = time_to_arrive(temp_total,temp_initial_total+(temp_stable_total-temp_initial_total)*0.63)
    
    HMC1_zoneA_63 = HMC1_zoneA_dynamic[time_initial_to_63_zoneA]
    HMC1_zoneB_63 = HMC1_zoneB_dynamic[time_initial_to_63_zoneB]
    
    list_time_initial_to_63_zoneA.append(time_initial_to_63_zoneA)
    list_time_initial_to_63_zoneB.append(time_initial_to_63_zoneB)
    list_time_initial_to_63_total.append(time_initial_to_63_total)
    list_HMC1_zoneA_63.append(HMC1_zoneA_63)
    list_HMC1_zoneB_63.append(HMC1_zoneB_63)
    
    TMC_zoneA_30min = temp_match_coefficient(temp_zoneA,temp_target,operating_duration_30min)
    TMC_zoneB_30min = temp_match_coefficient(temp_zoneB,temp_target,operating_duration_30min)
    TMC_total_30min = temp_match_coefficient(temp_total,temp_target,operating_duration_30min)
    TMC_zoneA_1h = temp_match_coefficient(temp_zoneA,temp_target,operating_duration_1h)
    TMC_zoneB_1h = temp_match_coefficient(temp_zoneB,temp_target,operating_duration_1h)
    TMC_total_1h = temp_match_coefficient(temp_total,temp_target,operating_duration_1h)
    TMC_zoneA_2h = temp_match_coefficient(temp_zoneA,temp_target,operating_duration_2h)
    TMC_zoneB_2h = temp_match_coefficient(temp_zoneB,temp_target,operating_duration_2h)
    TMC_total_2h = temp_match_coefficient(temp_total,temp_target,operating_duration_2h)
    TMC_zoneA_3h = temp_match_coefficient(temp_zoneA,temp_target,operating_duration_3h)
    TMC_zoneB_3h = temp_match_coefficient(temp_zoneB,temp_target,operating_duration_3h)
    TMC_total_3h = temp_match_coefficient(temp_total,temp_target,operating_duration_3h)    
    TMC_zoneA_4h = temp_match_coefficient(temp_zoneA,temp_target,operating_duration_4h)
    TMC_zoneB_4h = temp_match_coefficient(temp_zoneB,temp_target,operating_duration_4h)
    TMC_total_4h = temp_match_coefficient(temp_total,temp_target,operating_duration_4h)    
    TMC_zoneA_5h = temp_match_coefficient(temp_zoneA,temp_target,operating_duration_5h)
    TMC_zoneB_5h = temp_match_coefficient(temp_zoneB,temp_target,operating_duration_5h)
    TMC_total_5h = temp_match_coefficient(temp_total,temp_target,operating_duration_5h)    
    TMC_zoneA_6h = temp_match_coefficient(temp_zoneA,temp_target,operating_duration_6h)
    TMC_zoneB_6h = temp_match_coefficient(temp_zoneB,temp_target,operating_duration_6h)
    TMC_total_6h = temp_match_coefficient(temp_total,temp_target,operating_duration_6h)    

#   .............................................给定义变量赋值.................................................... 

    list_time_zoneA.append(time_initial_to_target_zoneA)
    list_time_zoneB.append(time_initial_to_target_zoneB)
    list_time_total.append(time_initial_to_target_total)
    
    #   0.5h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
    list_TMC_zoneA_30min.append(TMC_zoneA_30min)
    list_TMC_zoneB_30min.append(TMC_zoneB_30min)
    list_TMC_total_30min.append(TMC_total_30min)

    #   1h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
    list_TMC_zoneA_1h.append(TMC_zoneA_1h)
    list_TMC_zoneB_1h.append(TMC_zoneB_1h)
    list_TMC_total_1h.append(TMC_total_1h)

    #   2h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
    list_TMC_zoneA_2h.append(TMC_zoneA_2h)
    list_TMC_zoneB_2h.append(TMC_zoneB_2h)
    list_TMC_total_2h.append(TMC_total_2h)

    #   3h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
    list_TMC_zoneA_3h.append(TMC_zoneA_3h)
    list_TMC_zoneB_3h.append(TMC_zoneB_3h)
    list_TMC_total_3h.append(TMC_total_3h)

    #   4h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
    list_TMC_zoneA_4h.append(TMC_zoneA_4h)
    list_TMC_zoneB_4h.append(TMC_zoneB_4h)
    list_TMC_total_4h.append(TMC_total_4h)

    #   5h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
    list_TMC_zoneA_5h.append(TMC_zoneA_5h)
    list_TMC_zoneB_5h.append(TMC_zoneB_5h)
    list_TMC_total_5h.append(TMC_total_5h)

    #   6h内与室内外温差15℃目标匹配度：zoneA、zoneB、整体
    list_TMC_zoneA_6h.append(TMC_zoneA_6h)
    list_TMC_zoneB_6h.append(TMC_zoneB_6h)
    list_TMC_total_6h.append(TMC_total_6h)

#   .............................................不确定度计算.................................................... 
    
    W_temp_env = 0.1       #℃
    W_temp_ina = 0.1       #℃
    W_temp_tersur = 0.1    #℃
    W_temp_wat = 0.1       #℃
    W_airspeed = 0.05       #m/s
    W_waterflow = 0.005       #kg/h
    
    W_operative_demandzone = ((W_temp_env/2)**2 + (W_temp_ina/2)**2)**0.5
    W_operative_whole = ((W_temp_env/2)**2 + (W_temp_ina/2)**2)**0.5

    temp_up_whole = [(temp_up_total_01m[i]+temp_up_total_02m[i]+temp_up_total_03m[i]+temp_up_total_04m[i]+temp_up_total_05m[i]+temp_up_total_06m[i]+temp_up_total_07m[i]+temp_up_total_08m[i]+temp_up_total_09m[i]+temp_up_total_10m[i]+temp_up_total_11m[i]+temp_up_total_12m[i]+temp_up_total_13m[i]+temp_up_total_13m[i]+temp_up_total_14m[i]+temp_up_total_15m[i]+temp_up_total_16m[i]+temp_up_total_17m[i]+temp_up_total_18m[i]+temp_up_total_19m[i]+temp_up_total_20m[i]+temp_up_total_21m[i]+temp_up_total_22m[i]+temp_up_total_23m[i]+temp_up_total_24m[i]+temp_up_total_25m[i])/25 for i in range(len(temp_up_total_11m))]
    temp_up_demandzone = [(temp_up_total_01m[i]+temp_up_total_02m[i]+temp_up_total_03m[i]+temp_up_total_04m[i]+temp_up_total_05m[i]+temp_up_total_06m[i]+temp_up_total_07m[i]+temp_up_total_08m[i]+temp_up_total_09m[i]+temp_up_total_10m[i]+temp_up_total_11m[i]+temp_up_total_12m[i]+temp_up_total_13m[i]+temp_up_total_13m[i])/13 for i in range(len(temp_up_total_01m))]
    
    W_HMC63 = (2*(W_operative_demandzone/(temp_up_whole[time_initial_to_63_zoneA]))**2+2*(W_operative_whole*(temp_up_demandzone[time_initial_to_63_zoneA])/(temp_up_whole[time_initial_to_63_zoneA])**2)**2)**0.5
    W_HMC95 = (2*(W_operative_demandzone/(temp_up_whole[int(np.floor(time_stable_tersur*3600/10))]))**2+2*(W_operative_whole*(temp_up_demandzone[int(np.floor(time_stable_tersur*3600/10))])/(temp_up_whole[int(np.floor(time_stable_tersur*3600/10))])**2)**2)**0.5
    
    c_w = 4200      #J/kgK
   
    W_heat_tersur_dynamic = [((c_w*(temp_suwat[i]-temp_rewat[i])*W_waterflow)**2+(c_w*volume_suwat[i]/3600*W_temp_wat)**2+(c_w*volume_suwat[i]/3600*W_temp_wat)**2)**0.5 for i in range(len(temp_suwat))]
    W_heat_tersur = np.mean([W_heat_tersur_dynamic[i] for i in range(int(np.floor(end_time-3600/sta_duration)),end_time-1)])

    list_W_HMC63.append(W_HMC63)
    list_W_HMC95.append(W_HMC95)
    list_W_heat_tersur.append(W_heat_tersur)
    
    
    #   动态输出表格
    target_sheet2 = "211227_R1_50_0m.xlsx"
    if my_list[id_sheet1-1] == target_sheet2:
        temp_suwat = sheet1_temp_suwat[id_sheet1-1]
        temp_tersur = sheet1_temp_tersur[id_sheet1-1]
        temp_outair = sheet1_temp_outair[id_sheet1-1]
        props = ["temp_suwat","temp_tersur","temp_outair","temp_envelope",
                 "temp_zoneA","temp_zoneB","temp_total","temp_zoneC","temp_zoneD",
                 "HMC1_zoneA_dynamic","HMC1_zoneB_dynamic","HMC1_zoneC_dynamic","HMC1_zoneD_dynamic"]
        data_df = pd.DataFrame([temp_suwat,temp_tersur,temp_outair,temp_envelope,
                                temp_zoneA,temp_zoneB,temp_total,temp_zoneC,temp_zoneD,
                                HMC1_zoneA_dynamic,HMC1_zoneB_dynamic,HMC1_zoneC_dynamic,HMC1_zoneD_dynamic]).transpose()
        if len(data_df.index) > len(time):
            data_df = data_df.drop(data_df.index[len(time):])
        data_df.columns = props
        data_df.index = time
        with pd.ExcelWriter("results_dynamic/211227_R1_50_0m_dynamic.xlsx",engine="openpyxl") as writer:
            data_df.to_excel(writer,float_format='%.4f')


In [19]:
props1 = ['供水温度/℃','回水温度/℃',
          '末端表面平均温度/℃','末端表面温度1/℃','末端表面温度2/℃','末端表面温度3/℃','末端表面温度4/℃',
          '室内平均空气温度/℃','室外平均温度/℃','非供暖表面平均温度/℃',
          '东墙温度1/℃','东墙温度2/℃','西墙温度1/℃','西墙温度2/℃','北墙温度/℃','南墙温度/℃',
          '屋顶温度/℃','外窗温度/℃','地面温度/℃',
          '操作温度/℃',
          '热源供热量（末端自身特征）/W','热源供热量（末端对比归一）/W','末端表面供热量（末端自身特征）/W','末端表面供热量（末端对比归一）/W',
          '末端表面稳定时间/h','室内空气稳定时间/h',
          '最后一小时温升/℃',
          'R1/K/W','R2/K/W','C/J/K',
          '供热量：启动阶段热源累积/kWh','供热量：启动阶段末端向室内累积/kWh','蓄热量：启动阶段末端蓄热累积/kWh',
          '火积耗散：启动阶段热源-末端累积/kWh·K','火积耗散：启动阶段末端-室内累积/kWh·K','火积耗散：启动阶段室内-室外累积/kWh·K','火积耗散：启动阶段总累积/kWh·K',
          '火积耗散：稳定阶段热源-末端平均/WK','火积耗散：稳定阶段末端-室内平均/WK','火积耗散：稳定阶段室内-室外平均/WK','火积耗散：稳定阶段总平均/WK']

props2 = ['zoneA初始温度/℃','zoneB初始温度/℃','zoneC初始温度/℃','zoneD初始温度/℃',
          'zoneA稳定温度/℃','zoneB稳定温度/℃','zoneC稳定温度/℃','zoneD稳定温度/℃','1.7-0.1m温差','1.1-0.1m温差',
          'HMC_zoneA0.5h(用温升算)','HMC_zoneB0.5h(用温升算)','HMC_zoneC0.5h(用温升算)','HMC_zoneD0.5h(用温升算)',
          'HMC_zoneA_63%','HMC_zoneB_63%',
          'HMC_zoneA启动阶段(用温升算)','HMC_zoneB启动阶段(用温升算)','HMC_zoneC启动阶段(用温升算)','HMC_zoneD启动阶段(用温升算)',
          'HMC_zoneA启动阶段(用室内外温差算)','HMC_zoneB启动阶段(用室内外温差算)','HMC_zoneA稳定阶段(用室内外温差算)','HMC_zoneB稳定阶段(用室内外温差算)',
          'zoneA达到设定温度所需时长/h','zoneB达到设定温度所需时长/h','整体达到设定温度所需时长/h',
          'zoneA达到63%温升所需时长/h','zoneB达到63%温升所需时长/h','整体达到63%温升所需时长/h',
          'TMC_zoneA_0.5h','TMC_zoneB_0.5h','TMC_总_0.5h',
          'TMC_zoneA_1h','TMC_zoneB_1h','TMC_总_1h',
          'TMC_zoneA_2h','TMC_zoneB_2h','TMC_总_2h',
          'TMC_zoneA_3h','TMC_zoneB_3h','TMC_总_3h',
          'TMC_zoneA_4h','TMC_zoneB_4h','TMC_总_4h',
          'TMC_zoneA_5h','TMC_zoneB_5h','TMC_总_5h',
          'TMC_zoneA_6h','TMC_zoneB_6h','TMC_总_6h',
          'HMC63不确定度','HMC95不确定度','末端供热不确定度']          

In [20]:
file_store_data1 = [list_temp_stable_suwat,list_temp_stable_rewat,
                    list_temp_stable_tersur,list_temp_stable_tersur1,list_temp_stable_tersur2,list_temp_stable_tersur3,list_temp_stable_tersur4,
                    list_temp_stable_inair,list_temp_stable_outair,list_temp_stable_envelope,
                    list_temp_stable_eastwall1,list_temp_stable_eastwall2,list_temp_stable_westwall1,list_temp_stable_westwall2,list_temp_stable_northwall,list_temp_stable_southwall,
                    list_temp_stable_roof,list_temp_stable_window,list_temp_stable_floor,
                    list_temp_stable_ope,
                    list_heat_stable_suwat1,list_heat_stable_suwat2,list_heat_stable_tersur1,list_heat_stable_tersur2,
                    list_time_stable_tersur,list_time_stable_inair,
                    list_temprise_ope_lasthour,
                    list_R1,list_R2,list_C,
                    list_cumheat_start_suwat,list_cumheat_start_tersur,list_cumheat_start_store,
                    list_cumentransy_start_suwat_to_tersur,list_cumentransy_start_tersur_to_in,list_cumentransy_start_in_to_out,list_cumentransy_total,
                    list_entransy_stable_suwat_to_tersur,list_entransy_stable_tersur_to_in,list_entransy_stable_in_to_out,list_entransy_total]

file_store_data2 = [list_temp_initial_zoneA,list_temp_initial_zoneB,list_temp_initial_zoneC,list_temp_initial_zoneD,
                    list_temp_stable_zoneA,list_temp_stable_zoneB,list_temp_stable_zoneC,list_temp_stable_zoneD,list_tempdif_stable_17to01,list_tempdif_stable_11to01,
                    list_HMC1_zoneA_30min,list_HMC1_zoneB_30min,list_HMC1_zoneC_30min,list_HMC1_zoneD_30min,
                    list_HMC1_zoneA_63,list_HMC1_zoneB_63,
                    list_HMC1_zoneA_start,list_HMC1_zoneB_start,list_HMC1_zoneC_start,list_HMC1_zoneD_start,
                    list_HMC2_zoneA_start,list_HMC2_zoneB_start,list_HMC2_zoneA_stable,list_HMC2_zoneB_stable,
                    list_time_zoneA,list_time_zoneB,list_time_total,
                    list_time_initial_to_63_zoneA,list_time_initial_to_63_zoneB,list_time_initial_to_63_total,
                    list_TMC_zoneA_30min,list_TMC_zoneB_30min,list_TMC_total_30min,
                    list_TMC_zoneA_1h,list_TMC_zoneB_1h,list_TMC_total_1h,
                    list_TMC_zoneA_2h,list_TMC_zoneB_2h,list_TMC_total_2h,
                    list_TMC_zoneA_3h,list_TMC_zoneB_3h,list_TMC_total_3h,
                    list_TMC_zoneA_4h,list_TMC_zoneB_4h,list_TMC_total_4h,
                    list_TMC_zoneA_5h,list_TMC_zoneB_5h,list_TMC_total_5h,
                    list_TMC_zoneA_6h,list_TMC_zoneB_6h,list_TMC_total_6h,
                    list_W_HMC63,list_W_HMC95,list_W_heat_tersur]

In [21]:
data_df1 = pd.DataFrame(file_store_data1)
data_df1.columns = my_list
data_df1.index = props1

In [22]:
data_df2 = pd.DataFrame(file_store_data2)
data_df2.columns = my_list
data_df2.index = props2


In [23]:
with pd.ExcelWriter("Radiator_left_15℃_air.xlsx",engine="openpyxl") as writer:
    data_df1.to_excel(writer,float_format='%.4f',sheet_name='时间特征')
    data_df2.to_excel(writer,float_format='%.4f',sheet_name='空间特征')